In [13]:
import sys
import os
import re
import string
import time
import math
import psutil
import warnings
from contextlib import contextmanager

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm as tqdm

sys.path.append('../../')
from tokenize_ja import tokenizer

In [14]:
%load_ext autoreload
%autoreload 2
tqdm.pandas()
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', 600)
pd.options.display.float_format = '{:.5f}'.format

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
def clean_hashtag(text: str) -> str:
    """ハッシュタグを除外する
    Args:
        text (str): [description]
    Returns:
        str: [description]
    """
    text = re.sub(r'( #[a-zA-Z]+)+$', '', text)
    text = re.sub(r' #([a-zA-Z]+) ', r'\1', text)
    return text

In [16]:
text = '機械学習やるならpythonがいいよね。 #jupyter #pycon #scipy'

In [17]:
clean_hashtag(text)

'機械学習やるならpythonがいいよね。'

In [19]:
! sudachipy link -t full

Set the default dictionary to `sudachidict_full`.


In [20]:
sudachi = tokenizer.SudachiTokenizer(mode="A")

In [3]:
@contextmanager
def trace(title):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    yield
    m1 = p.memory_info()[0] / 2. ** 30
    delta = m1 - m0
    sign = '+' if delta >= 0 else '-'
    delta = math.fabs(delta)
    print(f"[{m1:.1f}GB({sign}{delta:.1f}GB):{time.time() - t0:.1f}sec] {title} ", file=sys.stderr)

In [33]:
def load_dataset(filename, n=5000, state=6):

    def filter_by_ascii_rate(text, threshold=0.9):
        ascii_letters = set(string.printable)
        rate = sum(c in ascii_letters for c in text) / len(text)
        return rate <= threshold

    df = pd.read_csv(filename, sep='\t')

    # extracts Japanese texts.
    is_jp = df.review_body.apply(filter_by_ascii_rate)
    df = df[is_jp]

    # sampling.
    df = df.sample(frac=1, random_state=state)  # shuffle
    grouped = df.groupby('star_rating')
    df = grouped.head(n=n)
    return pd.DataFrame(df).reset_index(drop='True')


def train_and_eval(x_train, y_train, x_test, y_test):
    vectorizer = CountVectorizer()
    x_train_vec = vectorizer.fit_transform(x_train)
    x_test_vec = vectorizer.transform(x_test)
    clf = LogisticRegression(solver='liblinear')
    clf.fit(x_train_vec, y_train)
    y_pred = clf.predict(x_test_vec)
    score = accuracy_score(y_test, y_pred)
    print('{:.4f}'.format(score))

In [34]:
with trace('load data'):
    df = load_dataset('data/amazon_reviews_multilingual_JP_v1_00.tsv', n=1000)

[1.6GB(+0.0GB):12.2sec] load data 


In [35]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,JP,36009604,RIEXG3Q0NCO1N,B00005LMCL,49084163,エリン・ブロコビッチ [DVD],Video DVD,5,1,1,N,Y,勝気な女性の,勝気な女性の、しかし、勝気な我侭な・・・じゃなくて、必死になれる献身さが反面にある。ほんとに、人間らしい女性のサクセスストーリーなんだけど、利他的な努力のできる素敵な人格の物語・・・と言うより、実話だそうです。実際、ジュリア・ロバーツだからできた映画かもしれません。,2013-08-13
1,JP,9801356,R2MFOSGN7O7V1V,B00BKTFABE,355356902,KindleのビデオカメラHD（ハードタブレット版）,Mobile_Apps,1,3,6,N,Y,クソ 最悪,裏切られました！ビデオ撮影ができなくなりアプリ停止<br />笑えてくる位ゴミ,2013-09-28
2,JP,48706054,R1WR5E7LXQFDVZ,B000025078,923583947,A Momentary Lapse of Reason,Music,5,9,11,N,N,デイヴギルモアズピンクフロイドの第１作・・・世界的に売れるだけのクォリティ！！,１９８７年リリースの新生ピンクフロイドの第１作・・・デイヴギルモアズピンクフロイドという形容がピッタリのアルバムで、内容は素晴らしく個人的には大好きです。基本的にはデイヴとプロデューサー・ボブエズリンが仕上げたものと言えます・・・ニックメイソンも完全に復帰しておらず（ドラムが叩けない・・・笑）、リチャードライトもサポート程度の扱いになっています。豪華ミュージシャンが参加しており、トニーレビン（Ｂ・Ｓｔｉｃｋ）・ジムケルトナー（Ｄｒ）・カーマインアピス（Ｄｒ）・マイケルランドゥ（Ｇ）・トムスコット（Ｓａｘ）等と錚々たる面子です・・・金かかってます（笑）！！ジャケットも合成ではなく実際ベッドを並べたとか・・・ストームソーガソンの仕事は本当スケールがでかい。４曲目「ＯＮＥＳＬＩＰ」はデイヴとフィルマンザネラとの共作で最高にかっこいい・・・トニーレビンが大活躍の曲！！５曲目「オンザターニングザウェイ」はデイヴの泣きのギターが炸裂する曲で名演です！！この時期の来日公演を体験しましたが、本当凄いバンドでした・・・とてつもないエネルギーを誇るロックバンド！！必聴！！！！！,2002-03-15
3,JP,26935613,R3C2EZ5WGPEI9U,B00IM0PZKK,126004443,ゼロ・グラビティ 3D & 2D ブルーレイセット(初回限定生産)2枚組 [Blu-ray],Video DVD,5,4,12,N,Y,こんな映画,見たことない。<br />ヘッドマウントディスプレイで見たのですが 息ができなくなりました。,2014-05-06
4,JP,28401471,R1WQCGH6NOCEOE,B002UD56HW,956054538,Michael Jackson's This Is It [UMD] [Import],Video DVD,4,2,4,N,Y,作品は最高!…だが,日本のPSPに対応してないので<br />コレクションアイテムになりましたw<br /><br />けど、コレクターとしては<br />いい品です(*'▽`)♪<br />メモリアルDVDBOXを購入してたので<br />今回の買い物は問題なしです♪,2010-01-26


In [36]:
df.shape

(5000, 15)

In [37]:
# 特殊な文字列が入っているレビューを除外
df = df[~df['review_body'].str.contains('ã')].reset_index(drop=True)

In [38]:
df.shape

(4995, 15)

In [39]:
X = df[['review_body']]
y = df[['star_rating']]

In [40]:
X

,review_body
0,勝気な女性の、しかし、勝気な我侭な・・・じゃなくて、必死になれる献身さが反面にある。ほんとに、人間らしい女性のサクセスストーリーなんだけど、利他的な努力のできる素敵な人格の物語・・・と言うより、実話だそうです。実際、ジュリア・ロバーツだからできた映画かもしれません。
1,裏切られました！ビデオ撮影ができなくなりアプリ停止<br />笑えてくる位ゴミ
2,１９８７年リリースの新生ピンクフロイドの第１作・・・デイヴギルモアズピンクフロイドという形容がピッタリのアルバムで、内容は素晴らしく個人的には大好きです。基本的にはデイヴとプロデューサー・ボブエズリンが仕上げたものと言えます・・・ニックメイソンも完全に復帰しておらず（ドラムが叩けない・・・笑）、リチャードライトもサポート程度の扱いになっています。豪華ミュージシャンが参加しており、トニーレビン（Ｂ・Ｓｔｉｃｋ）・ジムケルトナー（Ｄｒ）・カーマインアピス（Ｄｒ）・マイケルランドゥ（Ｇ）・トムスコット（Ｓａｘ）等と錚々たる面子です・・・金かかってます（笑）！！ジャケットも合成ではなく実際ベッドを並べたとか・・・ストームソーガソンの仕事は本当スケールがでかい。４曲目「ＯＮＥＳＬＩＰ」はデイヴとフィルマンザネラとの共作で最高にかっこいい・・・トニーレビンが大活躍の曲！！５曲目「オンザターニングザウェイ」はデイヴの泣きのギターが炸裂する曲で名演です！！この時期の来日公演を体験しましたが、本当凄いバンドでした・・・とてつもないエネルギーを誇るロックバンド！！必聴！！！！！
3,見たことない。<br />ヘッドマウントディスプレイで見たのですが 息ができなくなりました。
4,日本のPSPに対応してないので<br />コレクションアイテムになりましたw<br /><br />けど、コレクターとしては<br />いい品です(*'▽`)♪<br />メモリアルDVDBOXを購入してたので<br />今回の買い物は問題なしです♪
...,...
4990,デビュー以来のリアルタイムのリスナーですが、一番嫌いな時期に戻っちった、が素直な感想。そうメタル・ジャスティスだよ。カークのソロもそっくりだし。曲構成のバラバラなとこも。前作は評価されてない程嫌いじゃないけど、これは嫌。けど、メタル・ジャスティスの後にブラックアルバムが出たことに期待を込めて。次は
4991,冒頭の戦闘シーンは、さすがリドリー・スコット。緊張感に包まれた戦士の鼓動や戦場の匂いまでが伝わってくるような迫力。それにしても、この監督は登場人物が濡れる状況が好きだなあ。戦闘が終わると、脚本の瑕疵が気になりだしてしかたがなかった。反逆罪で殺されそうになった将軍が簡単に逃げられたのはなぜか？ 必死で家に戻った将軍よりも家族が虐殺されたのが先なのはなぜか？ 奴隷となったいろいろな民族が自由にコミュニケーションできたのは何語で会話していたのか？ 数え出すときりがないくらいに疑問が多い。頭の中に浮かぶクエスチョンマークをより分けながら観ていたが、同様にローマ時代を描いた『ベンハー』には遠く及ばない駄作ではないか。どうしちゃったんだリドリー・スコット。
4992,なんて内容がないんだ。<br /><br />楽しい絵本のような感じでしょうか。<br /><br />「みんなでうちに帰ろう〜」みたいなＣＭコピーでしたが、帰りませんから。<br /><br />頑張って帰るのかと思いきや、全然違うオチです。<br /><br />びっくりします。<br /><br />ＣＧ映画はおもしろいと思っていましたが、考え直しが必要です。
4993,つまらなかった。アギレラは好きだし、アギレラのPVも好きだが、映画として余りにもお粗末な内容だ。ストーリーも陳腐だし、アギレラの演技もセリフ棒読みで見ちゃいられない。やっぱりアギレラはミュージシャンであり、女優では無い、というのを実感させられる作品。歌のシーンも、妙にリアリティだしちゃって、PVのような迫力も無し。カメラワークも退屈だし、撮影監督誰だ?て感じ。アギレラにとっては、キャリアの汚点と言える作品では無いでしょうか。アギレラのファンは喜んで見るだろうけど。ミュージシャンものの映画って大体こんなんよね。期待が大きかっただけにガッカリな内容でした。<br />アギレラとシェールのギャラ高すぎて制作費に金を掛けられなかったか?


In [41]:
y

,star_rating
0,5
1,1
2,5
3,5
4,4
...,...
4990,2
4991,2
4992,2
4993,2


# tokenize

In [42]:
mecab_tokenizer = tokenizer.MeCabTokenizer(sys_dic_path='/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

In [44]:
with trace("mecab tokenize"):
    X.loc[:, 'mecab_token'] = X['review_body'].progress_apply(mecab_tokenizer.tokenize)

  0%|          | 0/4995 [00:00<?, ?it/s]

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
[2.1GB(+0.0GB):5.5sec] mecab tokenize 


In [45]:
X.head()

,review_body,mecab_token
0,勝気な女性の、しかし、勝気な我侭な・・・じゃなくて、必死になれる献身さが反面にある。ほんとに、人間らしい女性のサクセスストーリーなんだけど、利他的な努力のできる素敵な人格の物語・・・と言うより、実話だそうです。実際、ジュリア・ロバーツだからできた映画かもしれません。,"[勝気, 女性, 勝気, 我侭, じゃなくて, 必死, なれる, 献身, さ, 反面, ある, 人間らしい, 女性, サクセスストーリー, ん, 利他, 的, 努力, できる, 素敵, 人格, 物語, 言う, 実話, そう, ジュリア・ロバーツ, できる, 映画, しれる]"
1,裏切られました！ビデオ撮影ができなくなりアプリ停止<br />笑えてくる位ゴミ,"[裏切る, れる, ビデオ撮影, できる, なる, アプリ, 停止, BR, 笑える, くる, 位, ゴミ]"
2,１９８７年リリースの新生ピンクフロイドの第１作・・・デイヴギルモアズピンクフロイドという形容がピッタリのアルバムで、内容は素晴らしく個人的には大好きです。基本的にはデイヴとプロデューサー・ボブエズリンが仕上げたものと言えます・・・ニックメイソンも完全に復帰しておらず（ドラムが叩けない・・・笑）、リチャードライトもサポート程度の扱いになっています。豪華ミュージシャンが参加しており、トニーレビン（Ｂ・Ｓｔｉｃｋ）・ジムケルトナー（Ｄｒ）・カーマインアピス（Ｄｒ）・マイケルランドゥ（Ｇ）・トムスコット（Ｓａｘ）等と錚々たる面子です・・・金かかってます（笑）！！ジャケットも合成ではなく実際ベッドを並べたとか・・・ストームソーガソンの仕事は本当スケールがでかい。４曲目「ＯＮＥＳＬＩＰ」はデイヴとフィルマンザネラとの共作で最高にかっこいい・・・トニーレビンが大活躍の曲！！５曲目「オンザターニングザウェイ」はデイヴの泣きのギターが炸裂する曲で名演です！！この時期の来日公演を体験しましたが、本当凄いバンドでした・・・とてつもないエネルギーを誇るロックバンド！！必聴！！！！！,"[1987年, リリース, 新生, PINK FLOYD, 1, 作, ・・デイヴギルモアズピンクフロイド, 形容, アルバム, 内容, 素晴らしい, 個人的, 大好き, 基本的, デイヴ, プロデューサー, ボブ・エズリン, 仕上げる, もの, 言える, ・・ニックメイソン, 完全, 復帰, する, おる, ドラム, 叩ける, 笑, リチャード・ライト, サポート, 程度, 扱い, なる, いる, 豪華, ミュージシャン, 参加, する, おる, Tony Levin, b, stick, ジム・ケルトナー, DR, カーマイン・アピス, DR, Michael Landau, g, トム・スコット, sax, 等, 錚々たる, 面子, 金, かかる, 笑, ジャケット, 合成, ベッド, 並べる, ストーム・ソーガソン, 仕事, 本当, スケール, でかい, 4曲, 目, oneslip, デイヴ, フィル・マンザネラ, 共作, 最高, かっこいい, Tony Levin, 活躍, 曲, 5曲, 目, オンザターニングザウェイ, デイヴ, 泣き, ギター, 炸裂, する, 曲, 名演, 時期, 来日公演, 体験, する, 本当, 凄い, バンド, 途轍, ない, エネルギー, 誇る, ロックバンド, 必聴]"
3,見たことない。<br />ヘッドマウントディスプレイで見たのですが 息ができなくなりました。,"[見る, こと, ない, BR, ヘッドマウントディスプレイ, 見る, の, 息, できる, なる]"
4,日本のPSPに対応してないので<br />コレクションアイテムになりましたw<br /><br />けど、コレクターとしては<br />いい品です(*'▽`)♪<br />メモリアルDVDBOXを購入してたので<br />今回の買い物は問題なしです♪,"[日本, PSP, 対応, する, てる, BR, コレクション, アイテム, なる, w, BR, BR, コレクター, BR, いい, 品, BR, メモリアル, dvdbox, 購入, する, てる, BR, 今回, 買い物, 問題]"


In [55]:
X.loc[:, 'mecab_token'] = X['mecab_token'].apply(lambda x: ' '.join(x))

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [56]:
X

,review_body,mecab_token
0,勝気な女性の、しかし、勝気な我侭な・・・じゃなくて、必死になれる献身さが反面にある。ほんとに、人間らしい女性のサクセスストーリーなんだけど、利他的な努力のできる素敵な人格の物語・・・と言うより、実話だそうです。実際、ジュリア・ロバーツだからできた映画かもしれません。,勝気 女性 勝気 我侭 じゃなくて 必死 なれる 献身 さ 反面 ある 人間らしい 女性 サクセスストーリー ん 利他 的 努力 できる 素敵 人格 物語 言う 実話 そう ジュリア・ロバーツ できる 映画 しれる
1,裏切られました！ビデオ撮影ができなくなりアプリ停止<br />笑えてくる位ゴミ,裏切る れる ビデオ撮影 できる なる アプリ 停止 BR 笑える くる 位 ゴミ
2,１９８７年リリースの新生ピンクフロイドの第１作・・・デイヴギルモアズピンクフロイドという形容がピッタリのアルバムで、内容は素晴らしく個人的には大好きです。基本的にはデイヴとプロデューサー・ボブエズリンが仕上げたものと言えます・・・ニックメイソンも完全に復帰しておらず（ドラムが叩けない・・・笑）、リチャードライトもサポート程度の扱いになっています。豪華ミュージシャンが参加しており、トニーレビン（Ｂ・Ｓｔｉｃｋ）・ジムケルトナー（Ｄｒ）・カーマインアピス（Ｄｒ）・マイケルランドゥ（Ｇ）・トムスコット（Ｓａｘ）等と錚々たる面子です・・・金かかってます（笑）！！ジャケットも合成ではなく実際ベッドを並べたとか・・・ストームソーガソンの仕事は本当スケールがでかい。４曲目「ＯＮＥＳＬＩＰ」はデイヴとフィルマンザネラとの共作で最高にかっこいい・・・トニーレビンが大活躍の曲！！５曲目「オンザターニングザウェイ」はデイヴの泣きのギターが炸裂する曲で名演です！！この時期の来日公演を体験しましたが、本当凄いバンドでした・・・とてつもないエネルギーを誇るロックバンド！！必聴！！！！！,1987年 リリース 新生 PINK FLOYD 1 作 ・・デイヴギルモアズピンクフロイド 形容 アルバム 内容 素晴らしい 個人的 大好き 基本的 デイヴ プロデューサー ボブ・エズリン 仕上げる もの 言える ・・ニックメイソン 完全 復帰 する おる ドラム 叩ける 笑 リチャード・ライト サポート 程度 扱い なる いる 豪華 ミュージシャン 参加 する おる Tony Levin b stick ジム・ケルトナー DR カーマイン・アピス DR Michael Landau g トム・スコット sax 等 錚々たる 面子 金 かかる 笑 ジャケット 合成 ベッド 並べる ストーム・ソーガソン 仕事 本当 スケール でかい 4曲 目 oneslip デイヴ フィル・マンザネラ 共作 最高 かっこいい Tony Levin 活躍 曲 5曲 目 オンザターニングザウェイ デイヴ 泣き ギター 炸裂 する 曲 名演 時期 来日公演 体験 する 本当 凄い バンド 途轍 ない エネルギー 誇る ロックバンド 必聴
3,見たことない。<br />ヘッドマウントディスプレイで見たのですが 息ができなくなりました。,見る こと ない BR ヘッドマウントディスプレイ 見る の 息 できる なる
4,日本のPSPに対応してないので<br />コレクションアイテムになりましたw<br /><br />けど、コレクターとしては<br />いい品です(*'▽`)♪<br />メモリアルDVDBOXを購入してたので<br />今回の買い物は問題なしです♪,日本 PSP 対応 する てる BR コレクション アイテム なる w BR BR コレクター BR いい 品 BR メモリアル dvdbox 購入 する てる BR 今回 買い物 問題
...,...,...
4990,デビュー以来のリアルタイムのリスナーですが、一番嫌いな時期に戻っちった、が素直な感想。そうメタル・ジャスティスだよ。カークのソロもそっくりだし。曲構成のバラバラなとこも。前作は評価されてない程嫌いじゃないけど、これは嫌。けど、メタル・ジャスティスの後にブラックアルバムが出たことに期待を込めて。次は,デビュー以来 リアルタイム リスナー 一番 嫌い 時期 戻る ちる 素直 感想 そう メタル・ジャスティス QUIRK ソロ だし 曲 構成 バラバラ とこ 前作 評価 する れる てる 嫌い これ 嫌 メタル・ジャスティス 後 ブラック・アルバム 出る こと 期待 込める 次
4991,冒頭の戦闘シーンは、さすがリドリー・スコット。緊張感に包まれた戦士の鼓動や戦場の匂いまでが伝わってくるような迫力。それにしても、この監督は登場人物が濡れる状況が好きだなあ。戦闘が終わると、脚本の瑕疵が気になりだしてしかたがなかった。反逆罪で殺されそうになった将軍が簡単に逃げられたのはなぜか？ 必死で家に戻った将軍よりも家族が虐殺されたのが先なのはなぜか？ 奴隷となったいろいろな民族が自由にコミュニケーションできたのは何語で会話していたのか？ 数え出すときりがないくらいに疑問が多い。頭の中に浮かぶクエスチョンマークをより分けながら観ていたが、同様にローマ時代を描いた『ベンハー』には遠く及ばない駄作ではないか。どうしちゃったんだリドリー・スコット。,冒頭 戦闘シーン リドリー・スコット 緊張感 包む れる 戦士 鼓動 戦場 匂い 伝わる くる よう 迫力 それにしてもこの 監督 登場人物 濡れる 状況 好き 戦闘 終わる 脚本 瑕疵 気 なる だす しかた ない 反逆罪 殺す れる そう なる 将軍 簡単 逃げる られる の 必死 家 戻る 将軍 家族 虐殺 する れる の 先 の 奴隷 なる いろいろ 民族 自由 コミュニケーション できる の 何 語 会話 する いる の 数える 出す きり ない 疑問 多い 頭 中 浮かぶ クエスチョンマーク より分ける 観る いる 同様 ローマ 時代 描く ベンハー 遠い 及ぶ 駄作 する ちゃう ん リドリー・スコット
4992,なんて内容がないんだ。<br /><br />楽しい絵本のような感じでしょうか。<br /><br />「みんなでうちに帰ろう〜」みたいなＣＭコピーでしたが、帰りませんから。<br /><br />頑張って帰るのかと思いきや、全然違うオチです。<br /><br />びっくりします。<br /><br />ＣＧ映画はおもしろいと思っていましたが、考え直しが必要です。,内容 ない ん BR BR 楽しい 絵本 よう 感じ BR BR みんな うち 帰る みたい cm コピー 帰る BR BR 頑張る 帰る の 思う 違う オチ BR BR びっくり する BR BR CG 映画 おもしろい 思う いる 考え直す 必要
4993,つまらなかった。アギレラは好きだし、アギレラのPVも好きだが、映画として余りにもお粗末な内容だ。ストーリーも陳腐だし、アギレラの演技もセリフ棒読みで見ちゃいられない。やっぱりアギレラはミュージシャンであり、女優では無い、というのを実感させられる作品。歌のシーンも、妙にリアリティだしちゃって、PVのような迫力も無し。カメラワークも退屈だし、撮影監督誰だ?て感じ。アギレラにとっては、キャリアの汚点と言える作品では無いでしょうか。アギレラのファンは喜んで見るだろうけど。ミュージシャンものの映画って大体こんなんよね。期待が大きかっただけにガッカリな内容でした。<br />アギレラとシェールのギャラ高すぎて制作費に金を掛けられなかったか?,つまらない アギレラ 好き アギレラ pv 好きだ。 映画 粗末 内容 ストーリー 陳腐 アギレラ 演技 セリフ 棒読み 見る いる られる アギレラ ミュージシャン 女優 無い の 実感 する せる られる 作品 歌 シーン 妙 リアリティ ちゃう pv よう 迫力 無い カメラワーク 退屈 撮影監督 誰だ 感じ アギレラ キャリア 汚点 言える 作品 無い アギレラ ファン 見る ミュージシャン もの 映画 大体 困難 期待 大きい 内容 BR アギレラ シェール ギャラ 高 すぎ 制作 費 金 掛ける られる


# train test split

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
X_train.shape

(3996, 2)

In [59]:
y_train['star_rating'].values

array([3, 1, 3, ..., 3, 2, 5])

In [60]:
train_and_eval(X_train['mecab_token'], y_train['star_rating'].values, X_test['mecab_token'], y_test['star_rating'].values)

0.3443


# スクリプト実行

In [61]:
! python3 train_mecab.py

[0.4GB(+0.3GB):12.5sec] load data 
[1.0GB(+0.5GB):4.8sec] mecab tokenize 
0.3443
[1.0GB(+0.0GB):1.6sec] train 


In [2]:
# ! python3 train.py

Tokenization only.
0.4010
Clean html.
/usr/local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
0.4020
Normalize number.
0.3930
Base form.
0.3930
Lower text.
/usr/local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
0.3900
